In [ ]:
import os
import time
import numpy as np
import scipy

In [ ]:
def mat_2n(nvars):
#function mat = mat_2n(nvars)

# Outputs matrix with n rows, where the ith row contains 2^(n - i + 1)
# blocks of length 2^(i - 1), alternating between all 1's and all 0's.
# Each column corresponds to a truth assignment where a 1 in row i means
# that variable x_i is set to 1.

# Essentially the (0,1) Rademacher matrix of order n

    if nvars == 0:
        mat = np.array([[]], dtype=int)
    elif nvars == 1:
        mat = np.array([[1, 0]], dtype=int)
    else:
        mat_temp = mat_2n(nvars - 1)

        mat = np.concatenate([mat_temp, mat_temp], axis=1)
        last_row = np.concatenate([np.ones((1,2**(nvars-1)), dtype=int),np.zeros((1,2**(nvars-1)), dtype=int)], axis=1)
        mat = np.concatenate([mat, last_row], axis=0)
    return mat


In [ ]:
# Test Case
nvars = 3
mat = mat_2n(nvars)
print(mat)

[[1 0 1 0 1 0 1 0]
 [1 1 0 0 1 1 0 0]
 [1 1 1 1 0 0 0 0]]


In [ ]:
from scipy.special import comb

def count_signatures(nvars):
    # Counts the number of valid signatures for MBFs on nvars variables.
    # From OEIS, A007695. Modified c and kap to index starting from 1

    # Initialize/preallocate
    c = np.zeros((nvars + 1, comb(nvars, nvars // 2, exact=True) + 1), dtype=np.uint32)
    kap = np.zeros_like(c, dtype=np.uint32)
    kkk = np.zeros_like(c, dtype=np.uint32)
    c[0, 0] = 1
    c[0, 1] = 1
    kap[0, 0] = 0

    s = 2

    for r in range(nvars):
        d = s
        k = r
        j = 0
        s = 0

        x_max = comb(nvars, r+1, exact=True)

        for x in range(x_max + 1):
            kkk[r + 1, x] = k

            if x >= comb(k, r+1, exact=True):
                k = k + 1

            if x == 0:
                kap[r + 1, x] = 0
            else:
                kap[r + 1, x] = comb(k - 1, r, exact=True) + kap[r, x - comb(k - 1, r+1, exact=True)]

            while j < kap[r + 1, x]:
                d = d - c[r, j]
                j = j + 1

            c[r + 1, x] = d
            s = s + d

    return s, kap, c

In [ ]:
# Test Case
nvars = 3
s, kap, c = count_signatures(nvars)
print(f"S = {s}\n kap=\n{kap}\n c=\n{c}")

S = 10
 kap=
[[0 0 0 0]
 [0 1 1 1]
 [0 2 3 3]
 [0 3 0 0]]
 c=
[[1 1 0 0]
 [2 1 1 1]
 [5 2 1 1]
 [9 1 0 0]]


In [ ]:
def rows_to_bits(bytes):
    # Compresses the rows of a 0-1 matrix into an integer matrix whose bits give the old matrix.
    # The first column of the original matrix becomes the least significant bits of the first column of the new matrix, and so on.

    int_size = 32

    num_rows = bytes.shape[0]
    num_cols = bytes.shape[1]
    compact_cols = np.ceil(num_cols / 32).astype(int)
    last_length = num_cols - int_size * (compact_cols - 1)

    bits = np.zeros((num_rows, compact_cols), dtype=np.uint32)

    for j in range(compact_cols):
        last = int_size  # Last column to fill.
        if j == compact_cols - 1:
            last = last_length
        for k in range(last):
            line = bytes[:, int_size * j + k]
            bits[:, j] = np.bitwise_or(bits[:, j], np.left_shift(line, k))

    return bits


In [ ]:
# Test Case
bytes = np.array([[1, 0, 1, 0], [1, 1, 1, 0]])
bit = rows_to_bits(bytes)
print(bit)

[[5]
 [7]]


In [ ]:
def rfb(bits, nvars):
    # Uncompresses the rows of a matrix encoded as bits into a 0-1 integer matrix.
    # The first column of the compressed matrix encodes the first 32 columns of the new matrix, and so on.
    # The additional parameter nvars encodes the number of variables.

    int_size = 32                # Number of bits in an integer.
    num_rows = bits.shape[0]     # Number of rows of bit-encoded columns.
    compact_cols = bits.shape[1] # Number of input columns.

    last_length = nvars - (compact_cols - 1) * int_size

    # Decode.
    bytes = np.zeros((num_rows, nvars))
    for i in range(compact_cols):
        curr_col = bits[:, i]
        last_index = int_size

        if i == compact_cols - 1:
            last_index = last_length
        mask = np.zeros(num_rows, dtype=np.uint32)
        mask = np.bitwise_or(mask, 1)
        for j in range(last_index):
            bytes[:, int_size * i + j] = np.right_shift(np.bitwise_and(curr_col, mask), j)
            mask = np.left_shift(mask, 1)

    return bytes.astype('int32')


In [ ]:
# Test Case
bits = bit # It is taken from previous function
nvars = 3
bytes = rfb(bits, nvars)
print(bytes)

[[1 0 1]
 [1 1 1]]


In [ ]:
def perm(mat,sgn,dim):

# Permutes the rows or columns of the matrix mat according to the
# permutation sgn, a row vector containing the numbers from 1 to n.

# dim gives the dimension being permuted.
    if not isinstance(mat, np.ndarray):
        mat = np.asarray(mat)
    if not isinstance(sgn, np.ndarray):
        sgn = np.asarray(sgn)

    red = np.ones(sgn.shape[0], dtype=int)
    sgn = sgn - red
# Check for correct size, too.

    if dim == 1:
        if mat.shape[0] == len(sgn):
            return mat[sgn,:]
        else:
            print(f"Incorrect dimensions")
            return

    elif dim == 2:
        if mat.shape[1] == len(sgn):
            return mat[:,sgn]
        else:
            print(f"Incorrect dimensions")
            return
    else:
        print(f"Invalid argument for dimension. dim should be 1 or 2")
        return

In [ ]:
# Test Case
mat = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sgn = [1, 3, 2]
dim = 1
mat = perm(mat,sgn,dim)
print(mat)

[[1 2 3]
 [7 8 9]
 [4 5 6]]


In [ ]:
def mts(f, nvars):
    # Given a MBF f, this function outputs its minimal true statements.

    comp = mat_2n(nvars)
    result = np.zeros((nvars, 0))

    if not isinstance(f, np.ndarray):

      if f == 1:
          result = np.zeros((nvars, 1), dtype=np.int32)
          return result

    while np.any(np.where(f != 0)):
        x = np.ones(2**nvars, dtype=np.int32)

        curr_bit = np.where(f)[0][-1]
        col = comp[:, curr_bit]
        result = np.concatenate((result, col.reshape(-1, 1)), axis=1)

        ind = np.where(col)[0]
        card = len(ind)

        for k in range(card):
            x = np.bitwise_and(x, comp[ind[k], :])
        f = np.maximum(f - x, 0)

    return result.astype('int64')

In [ ]:
# Test Case
f = np.array([1, 0, 1, 1, 0, 1, 1, 0])
nvars = 3
res = mts(f, nvars)
print(res)

[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [ ]:
def phi_1(nvars):
#function result = phi_1(nvars)

# All monotone boolean functions in nvars variables with exactly one
# minterm. (Consequently, ith row is the shadow of the minterm of that
# function).
    result = np.zeros((0,2**nvars), dtype=int)
    comp = mat_2n(nvars)

    for k in range(2**nvars):
        col = comp[:,k]
        ind = np.where(col != 0)

        x = np.ones((1,2**nvars), dtype=int)

        for i in range(len(ind[0])):

            x = x & comp[ind[0][i],:]

        result = np.concatenate([result, x], axis=0)

    return result


In [ ]:
# Test Case
nvars = 3
res = phi_1(nvars)
print(res)

[[1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [1 0 0 0 1 0 0 0]
 [1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1]]


In [ ]:
def mts_to_mbf(M, nvars):
    # Given a matrix of minimal true statements, outputs the MBF f with these minterms.

    comp = mat_2n(nvars)
    result = np.zeros((1, 2**nvars), dtype=int)

    if not isinstance(f, np.ndarray):
      if M == 0:
          result = np.ones((1, 2**nvars), dtype=int64)
          return result

    total = M.shape[1]

    for i in range(total):
        curr_col = M[:, i]
        ind = np.where(curr_col)[0]
        card = ind.size

        temp = np.ones((1, 2**nvars), dtype=int)

        for k in range(card):
            temp = np.bitwise_and(temp, comp[ind[k], :])

        result = np.bitwise_or(result, temp)

    return result


In [ ]:
# Test Case
M = np.array([[1, 0, 1, 1, 0, 1, 1, 0]]) # This must be 2D numpy array
nvars = 3
res = mts_to_mbf(M, nvars)
print(res)

[[1 1 1 1 1 1 1 1]]


In [ ]:
def mbf_comp(M_in, nvars, D_1, comp):
    # Given a matrix of MBFs M_in (in expanded form), for each row f,
    # this function outputs all statements comparable to its minterms (from the top).

    last = M_in.shape[0]
    resultM = np.zeros((last, 2**nvars))
    result_rightM = resultM.copy()

    for i in range(last):
        f = M_in[i, :]

        result = f
        result_right = np.zeros(2**nvars)

        if np.array_equal(f, np.ones(2**nvars)):
            resultM[i, :] = np.ones(2**nvars)

        while np.any(f):
            x = np.ones(2**nvars)
            y = np.zeros(2**nvars)

            curr_bit = np.where(f)[0][-1]
            col = comp[:, curr_bit]

            x = D_1[curr_bit, :]
            ind2 = np.where(col == 0)[0]
            card2 = len(ind2)

            for j in range(card2):
                y = np.bitwise_or(y, comp[ind2[j], :])

            f = np.maximum(f - x, 0)
            result_right = np.bitwise_or(~y, result_right)

        resultM[i, :] = result
        result_rightM[i, :] = result_right

    return resultM, result_rightM

In [ ]:
print("Didn't Happed")
# Test Case
M_in= np.array([[1, 0, 1, 1, 0, 1, 1, 0]]) # This must be 2D numpy array
nvars = 3
D_1 = np.array([[1, 0, 1, 1, 0, 1, 1, 0]])
comp = np.array([[1, 0, 1, 1, 0, 1, 1, 0]])
resultM, result_rightM = mbf_comp(M_in, nvars, D_1, comp)
print(resultM, result_rightM)

Didn't Happed


IndexError: ignored

In [ ]:
def hash7b(f_in):

	# Computes the corresponding key of function f_in given in compressed form
	# Assumes f_in is a 7-variable MBF, with 4 32-bit components
	if not isinstance(f_in, np.ndarray):
		f_in = np.asarray(f_in)

	# Check if it has the correct length
	if f_in.shape[1] == 4:
		pass
	else:
		print('Function not the correct size.')


	# Method: Nonlinear Hashing (Polynomial)
	key = 0

	for i in range(4):
		key = (key + f_in[i][0]) % 19933999
		key = (key*33) % 19933999

	return key

In [ ]:
# Test Case
f_in = np.array([[1, 0, 1, 1], [1, 1, 0, 1], [1, 0, 1, 1], [1, 1, 0, 1]], dtype=np.int64)
print(hash7b(f_in))

1222980

In [ ]:
def findperm(f,sgn,nvars):

# INPUTS
# nvars: number of variables
# f: function of length 2^nvars
# sgn: permutation in S^n

# Permutes the boolean function f (given in 0-1 form, with length 2^nvars)
# according to sgn, a permutation of nvars variables.
    if not isinstance(f, np.ndarray):
        f = np.asarray(f)
    if f.ndim !=2:
            f = np.array([f])
    if not isinstance(sgn, np.ndarray):
        sgn = np.asarray(sgn)
    red = np.ones(sgn.shape[0], dtype=int)
    sgn = sgn - red

# Check dimensions
    if ((f.shape[1] == 2**nvars) and (len(sgn) == nvars)):
        pass
    else:
        print(f'One or more inputs are invalid.')
        return

    mat = mat_2n(nvars)
    mat_new = perm(mat, sgn, 1)

    nperm = np.ones((2**nvars), dtype=int)

    for k in range(2**nvars):
        col_temp = mat_new[:,k]

        mask = np.ones((2**nvars), dtype=int)

        for i in range(nvars):
            temp = np.logical_or(np.logical_and(col_temp[i],mat[i,:]),np.logical_not(np.logical_or(col_temp[i],mat[i,:])))
            mask = np.logical_and(mask, temp)

        nperm[k] = np.asarray(np.where(mask==True)).flatten()[0] + 1

    fperm = perm(f,nperm,2)

    return fperm

In [ ]:
# Test Case
mat = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
sgn = [1, 3, 2]
nvars = 3
per = findperm(mat,sgn,nvars)
print(per)

In [ ]:
def stepRhash(f, m, D_1, D_1_right, comp, perm_list):

    # INPUTS
    # f: MBF in n variables
    # m: integer less than n
    # D_1: all n-variable MBFs with one minimal term
    # D_1_right: 'complements' of D_1, but not used
    # comp: mat_2n(n), (0,1)-Rademacher matrix of order n
    # perm_list: list of permutations from permlist(nvars)
    # OUTPUT: All MBFs that could be possibly formed by adding a set of size m to f.
    # If no such function exists, exist = 0.

    # Initialize
    len_f = len(f)
    nvars = int(np.log2(len_f))
    setsize = np.sum(comp, axis=0)
    indices = (setsize == m)
    lengthf = 2 ** nvars

    M_perms2 = np.zeros((2, lengthf))

    if nvars > 5:
        c_cols = 2 ** (nvars - 5)
    else:
        c_cols = 1
    sorter_comp = np.arange(c_cols)

    result = np.zeros((0, c_cols), dtype=np.uint32)

    if np.all(f[indices]) == 1:
        exist = 0
        result = f
        return result, exist

    # Get all comparable terms to minimal terms of f
    _, reverse = mbf_comp(f, nvars, D_1, comp)

    # Terms which are okay to be added
    available = np.bitwise_not(np.bitwise_or(reverse, f))

    # Check if there are any m-sets available
    if np.any(np.bitwise_and(available, indices)) == 1:

        # If there are sets which can be added
        exist = 1

        # Get all these sets
        candidates = np.where(np.bitwise_and(available, indices))[0]
        num_cand = len(candidates)

        # Go through all candidates
        for k in range(num_cand):

            # Take conjunction with candidate term
            new_entry = np.bitwise_or(f, D_1[candidates[k], :])
            M_perms2[0, :] = new_entry

            # Generate all permutations
            for p in range(perm_list.shape[0]):
                M_perms2[1, :] = new_entry[perm_list[p, :]]

                # for each permuted function obtained, retain smaller one only
                check_ind = np.where(M_perms2[0, :] != M_perms2[1, :])[0][0]

                if M_perms2[1, check_ind] < M_perms2[0, check_ind]:
                    # Least representative is now the new entry
                    M_perms2[0, :] = M_perms2[1, :]

            M_perms_out = rows_to_bits(M_perms2)  # Compress
            M_new_least = M_perms_out[0, :]  # Get least representative

            result = np.vstack((result, M_new_least))  # Append to result matrix so far

    else:
        # If no set can be added
        exist = 0
        result = f

    return result, exist

In [ ]:
from itertools import permutations

def permlist(nvars):
    # function A = permlist(nvars)

    # Generates all permutations of n-variable BFs
    # looking at the truth table forms (so length 2^n instead of n)

    # First, generate all permutations in S^n
    lst = np.array(list(permutations(range(nvars, 0, -1))))
    A = None

    # Then permute the index vector 1:2^n corresponding to
    # all these permutations to get resulting
    # indices for each
    for i in range(lst.shape[0]):
        B = findperm(np.array(range(1,2**nvars+1)),lst[i,:],nvars).flatten()
        A = (np.vstack((A, B)) if (A is not None) else B)
    return A

In [ ]:
# Test Case
nvars = 3
a = permlist(nvars)
print(a)

In [ ]:
def mts_mat(mat, nvars):
    # Outputs result, the minimal true statements of the MBFs in the matrix mat, as a string.
    # Also outputs part, the profiles of each MBF in mat.

    num_row, num_col = mat.shape

    result = []
    part = np.zeros((num_row, nvars))
    part_o = []

    for i in range(num_row):
        if np.array_equal(mat[i, :], np.zeros(2**nvars)):
            result.append('none')
            part[i, :] = np.zeros(nvars)
            part_o.append(str(np.zeros(nvars)))
        elif np.array_equal(mat[i, :], np.ones(2**nvars)):
            result.append('empty set')
            part[i, :] = np.zeros(nvars)
            part_o.append('empty set')
        else:
            curr = mts(mat[i, :], nvars)

            ind = np.sum(curr, axis=0)
            part_o.append(str(ind))

            for k in range(nvars):
                part[i, k] = np.sum(ind == k+1)

            x, y = np.where(curr)
            temp = np.concatenate((x.reshape(-1, 1), y.reshape(-1, 1)), axis=1)

            result.append(str(temp[:, 0]))
            # print(result)

    # print(result,part_o)
    # result[0, :] = []
    # part_o[0, :] = []

    return result, part

In [ ]:
# Test Case
nvars = 3
mat = mat_2n(nvars)
# print(f"mat = \n{mat}")
res, par = mts_mat(mat, nvars)
print(f"Result={res}\n Part={par}")

Result=['[0]', '[1]', '[2]']
 Part=[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
def lookup7b(f_in,table):

	# Finds function f_in in hash table "table"
	# Outputs where = key if it is there, where = next empty position if not.
	# Outputs there = 1 if it is there, there = 0 if not
	if not isinstance(f_in, np.ndarray):
		f_in = np.asarray(f_in)
	if not isinstance(table, np.ndarray):
		table = np.asarray(table)

	# Checks sizes of inputs
	if table.shape[0] == 19933999:
		pass
	else:
		print(f'Error: hash table incorrect size, should be 19933999')
		return

	if table.shape[1] == 4:
		pass
	else:
		print(f'Error: hash table does not have 4 columns')
		return

	if f_in.shape[1] == 4:
		pass
	else:
		print(f'Error: function input should have 4 compressed integers')

	key = hash7b(f_in)

	if key == 0:
		key = 19933999

	found = 0

	while found == 0:
		if (table[key-1,:] == np.array([0,0,0,0])).all():
			found = 1
			where = key
			there = 0
		elif (table[key-1,:] == f_in).all():
			found = 1
			where = key
			there = 1
		else:
			key = (key + 1) % 19933999
			if key == 0:
				key = 19933999

	return there, where

In [ ]:
# Test Case
f_in = np.array([[1, 0, 1, 1], [1, 1, 0, 1], [1, 0, 1, 1], [1, 1, 0, 1]], dtype=np.int64)
table = np.array([[1, 0, 1, 1], [1, 1, 0, 1], [1, 0, 1, 1], [1, 1, 0, 1]], dtype=np.int64)
print(lookup7b(f_in,table))

Error: hash table incorrect size, should be 19933999
None


In [ ]:
from scipy.special import comb

def gen_profiles(nvars):
    # Generates all possible profiles of MBFs on n variables, using count_signatures.

    # Initialize; this includes all profiles up to r = 1
    result = np.vstack((np.zeros(nvars), np.transpose(np.vstack((np.arange(1, 3 + 1), np.zeros((3 - 1, 3))))))).astype('int32')

    # Keeps track of how many profiles we have so far
    total = nvars + 1

    # Count signatures using algorithm from A007695
    # Save intermediate matrices 'kap' and 'c'
    _, kap, c = count_signatures(nvars)

    for r in range(2, nvars + 1):

        x_max = comb(nvars, r, exact=True)

        num_new = c[r, 0] - c[r - 1, 0]  # Number of profiles from previous iteration
        index = [i for i in range(total-num_new, total)]
        # print(total)
        consider = result[index, :]  # Retrieve these profiles

        for x in range(x_max):

            K = kap[r, x+1]  # Gives number of (r-1)-sets that x amount of r-sets correspond to

            # Gives rows that satisfy criteria
            # i.e. having at least K (r-1)-sets
            new_entries = consider[consider[:, r - 2] >= K, :]

            # Make "substitution"
            new_entries[:, r - 2] = new_entries[:, r - 2] - K
            new_entries[:, r - 1] = new_entries[:, r - 1] + x + 1

            # Append to existing list and update total
            result = np.vstack((result, new_entries))
            total = total + new_entries.shape[0]

    return result

In [ ]:
# Test Case
nvars = 3
res = gen_profiles(nvars)
print(res)

[[0 0 0]
 [1 0 0]
 [2 0 0]
 [3 0 0]
 [0 1 0]
 [1 1 0]
 [0 2 0]
 [0 3 0]
 [0 0 1]]


# count_functions_R_deltasHashGen

In [ ]:
# from phi_1 import phi_1
# from mat_2n import mat_2n
# from mbf_comp import mbf_comp
# from mts_mat import mts_mat
# from rfb import rfb
# from stepRhash import stepRhash
# from lookup7b import lookup7b

def count_functions_R_deltasHashGen(prof, perm_list, starting_f):
    # Given prof, a matrix of profile vectors, counts the number of MBFs with those profiles
    # in n variables (where n = length of prof)

    # Requires that each succeeding row in prof have to differ from the
    # previous row by exactly one set.

    # Accepts input starting_f as a starting point.
    # This corresponds to the list of functions in R(n) with profile prof[0]

    # starting_f is assumed to be in 0-1 expanded form

    list_size, nvars = prof.shape

    # Initialize functions with 1 minterm & comparable ones
    D_1 = phi_1(nvars)
    comp = mat_2n(nvars)
    dummy2, D_1_right = mbf_comp(D_1, nvars, D_1, comp)

    # For compressing
    if nvars > 5:
        c_cols = 2 ** (nvars - 5)
    else:
        c_cols = 1

    sorter_comp = np.arange(1, c_cols + 1)

    # Check if input profiles really only have 1 set differences between consecutive rows
    deltas = prof[1:list_size, :] - prof[0:list_size-1, :]
    isone = np.sum(deltas, axis=1) == 1
    if np.all(isone):
        pass
    else:
        wrong = np.where(isone == 0)[0][0]
        print(f"Incorrect profile input at {wrong}")
        return

    # Check starting_f if accurate (get first 20 rows only if more than 20)
    if starting_f.shape[0] > 100:
        start_check = starting_f[0:100, :]
    else:
        start_check = starting_f

    dummy1, dummy2 = mts_mat(start_check, nvars)  # Get profiles

    # Check if they have the correct profile
    for k in range(nvars):
        if not np.array_equal(dummy2[:, k], prof[0, k]):
            invalid_ind = np.where(dummy2[:, k] != prof[0, k])[0][0]
            print(f"Function {invalid_ind} is invalid with profile {dummy2[invalid_ind, :]}")
            return

    result = np.zeros((list_size, 1), dtype=np.uint32)  # Will store output per profile
    start_time = time.process_time()  # Start time

    for i in range(list_size - 1):  # Start at 1 since we have i = 0 as input.
        # Take only deltas between rows. Get index directly.
        curr_size = np.where(deltas[i, :])[0]

        if i == 0:
            fcns_exp = starting_f
            result[0] = starting_f.shape[0]
            print(f"Initial profile {prof[i, :]} has {result[i]} functions.")
            print()
        else:
            fcns_exp = rfb(fcns, 2 ** nvars)
            # Write function to file
            fname = f"{prof[i, 1]}_{prof[i, 2]}_{prof[i, 3]}_{prof[i, 4]}.bin"
            with open(fname, "wb") as file:
                file.write(fcns_exp)
            # End write

        temp = np.zeros((19933999, c_cols), dtype=np.uint32)  # Initialize Hash Table

        for k in range(fcns_exp.shape[0]):
            dummy, exist = stepRhash(fcns_exp[k, :], curr_size, D_1, D_1_right, comp, perm_list)

            if exist == 1:
                for b in range(dummy.shape[0]):
                    # Lookup hash table if function is there
                    there, where = lookup7b(dummy[b, :], temp)
                    if there == 0:
                        temp[where, :] = dummy[b, :]  # If function is not there, insert

        fcns = temp[temp[:, 0] > 0, :]  # Get nontrivial entries in hash table, these are the functions
        result[i + 1] = fcns.shape[0]  # Record number obtained

        # Print output
        print(f"Profile {prof[i + 1, :]} done after {time.process_time() - start_time} seconds.")
        print(f"Number of functions: {result[i + 1]}")
        print()

    return result


In [ ]:
# Write last profile
fcns_exp = rfb(fcns, 2 ** nvars)
i = list_size - 1
fname = f"{prof[i, 1]}_{prof[i, 2]}_{prof[i, 3]}_{prof[i, 4]}.bin"
with open(fname, "wb") as file:
    file.write(fcns_exp)
# End write